# Dynamic Programming (Part 2)

In this notebook, we'll implemement dynamic programming algorithms for some more problems: Weighted Independent Set in a Tree and TSP.

##### If you're using Datahub:
* Run the cell below **and restart the kernel if needed**

##### If you're running locally:
* Make sure you've activated the conda environment: `conda activate cs170`
* Launch jupyter: `jupyter notebook` or `jupyter lab` 
* Run the cell below **and restart the kernel if needed**

In [3]:
# Install dependencies
!pip install -r requirements.txt --quiet

In [4]:
import otter
assert (otter.__version__ >= "4.4.1"), "Please reinstall the requirements and restart your kernel."

grader = otter.Notebook("dp2.ipynb")
import pickle
import numpy.random as random
import random
import tqdm
import time
from autograder_utils import is_independent_set, validate_tour, handle_timeout

with open('public_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

rng_seed = 0

## Q1. Weighted Independent Set in a Tree

In class, we saw an algorithm to solve the maximum independent set on trees. For a quick refresher, you can consult https://people.eecs.berkeley.edu/~vazirani/algorithms/chap6.pdf#page=21. 

In this problem, we'll add some twists: now, suppose that each vertex $i$ has some weight $w[i]$, and we want to find the independent set with the maximum total weight. **Also, you must implement your algorithm with a top-down approach.**

As before, the algorithm discussed in lecture and the textbook only returns the size of the maximum independent set. However, here we want you to return the list of vertices in the acual independent set. To find the actual independent set, it may be useful to maintain an array seperate from the DP array which can help us backtrack to reconstruct the actual set (simply storing the entire set so far at each entry in the DP table is too expensive and will cause the autograder to fail).

Hint: To compute a node's children and grandchildren, feel free to re-use your code from previous homeworks.

In [3]:
def max_independent_set(adjacency_list, weights):
    """
    Return a list containing the vertices in the maximum weighted independent set.

    args:
        adjacency_list:ListList[[int]] = the adjacency list of the tree. 
        weights:List[int] = a list of vertex weights. weights[i] is the weight of vertex i.

    return:
        List[int] Containing the labels of the vertices in the maximum weighted independent set.
    """
    n = len(adjacency_list)
    if n == 0:
        return []
    
    # helper function 
    def get_post(adj_list):
        time, post, visited = 1, [0] * n, [False] * n
        unvisited = lambda i: not visited[i]

        def explore(u):
            nonlocal time
            visited[u] = True
            time += 1
            for v in adj_list[u]:
                if unvisited(v):
                    explore(v)
            post[u] = time
            time += 1

        for u in range(n):
            if unvisited(u):
                explore(u)
        return post
    
    post = get_post(adjacency_list)

    def get_children(u):
        yield from filter(lambda v: post[v] < post[u], adjacency_list[u])

    def get_grandchildren(u):
        for v in get_children(u):
            yield from get_children(v)

    dp = [None] * n

    def independent_set_helper(u):
        if dp[u] is not None:
            return dp[u]
        # the first element of the tuple is the weight of the set, the second element is whether the set includes u
        include_self = (weights[u] + sum(independent_set_helper(v)[0] for v in get_grandchildren(u)), True)
        exclude_self = (sum(independent_set_helper(v)[0] for v in get_children(u)), False)
        dp[u] = max(include_self, exclude_self)
        return dp[u]
    
    root = 0
    independent_set_helper(root)

    # get the set of vertices in the maximum weighted independent set
    independent_set = []
    to_check = [root]
    while to_check:
        u = to_check.pop()
        if dp[u][1]:
            independent_set.append(u)
            to_check.extend(get_grandchildren(u))
        else:
            to_check.extend(get_children(u))
    return independent_set


### Debugging
A simplified verion of the otter tests are pasted here for your convenience. Feel free to add whatever print statements or assertions you'd like when debugging.

__Note: your solution should not take inordinate amounts of time to run. Our implementation runs on all local test cases in <1s, and your implementation should run in similar time.__

_Points:_ 3

In [4]:
for adj_list, weights, expected_weight in tqdm.tqdm(test_data['q1.1']):
    try:
        result = max_independent_set(adj_list, weights)
    except Exception as e:
        raise Exception(f"Your function threw an error (see above)") from e
    assert is_independent_set(adj_list, result), f"Your output is not an independent set"
    assert sum(weights[i] for i in result) == expected_weight, f"Your output is not a maximum independent set"

100%|██████████| 100/100 [00:00<00:00, 3037.72it/s]


In [5]:
grader.check("independent set")

100%|██████████| 30/30 [00:00<00:00, 74.48it/s]


independent set results: All test cases passed!

## Traveling Salesperson DP
Now, we'll implement the dynamic programming algorithm for the traveling salesperson problem (TSP). A brute force solution will be hopelessly slow even for moderate-sized test cases, but we can use dynamic programming to get a solution in slightly more reasonable (but still exponential) time. For a refresher on the TSP algorithm, see Lecture 14 or https://people.eecs.berkeley.edu/~vazirani/algorithms/chap6.pdf#page=20. 

As with previous problems, we want you to return the actual tour, not the cost of the tour. We can once again apply the same procedure of backtracking through our subproblem array to reconstruct this tour.

##### A note on implementation:
Since our subproblem definition takes in a set as one of its parameters, we can't just use a 2D array to store our subproblems. Instead, we recommend storing subproblems in a dictionary, where the keys are tuples of the form `(S, i)`, where `i` represents the last city visited before returning home and `S` is the set of cities visited so far. 

To ensure that the keys are hashable, we recommend using Python's built-in `frozenset` class for `S`. `frozenset` is built-in to Python so you can use it without any additional imports, and works just like a normal set, except that it is immutable and hashable. You can read more about `frozenset` here: https://docs.python.org/3/library/stdtypes.html#frozenset.

An alternative approach would be to use a 2D array as usual, but use a bitmask to represent the set of visited cities. In this approach, `S` would be represented as an $n$-bit unsigned integer, and the $i$-th bit of `S` would be set to 1 if and only if the $i$-th city is part of the set of visited cities. Then, since `S` is an integer, we can use it to index into our 2D array.

As with before, storing the entire tour at each step is too memory-intensive and will cause the autograder to fail. Instead, consider maintaining a separate dictionary or array which stores a smaller amount of information but can still help you reconstruct the tour.

In [5]:
def tsp_dp(dist_arr):
    """Compute the exact solution to the TSP using dynamic programming and returns the optimal path.

    Args:
        dist_arr (ndarray[int]]): An n x n matrix of distances between cities. dist_arr[i][j] is the distance from city i to city j.

    Returns:
        List[int]: A list of city indices representing the optimal path.
    """
    n = len(dist_arr)
    if n == 0:
        return []
    dp = {}
    prev = {}
    root = 0

    def tsp_helper(S, i):
        """
        Args:
            S (frozenset[int]): A frozenset of cities that have not been visited.
            i (int): The current city.
        """
        if (S, i) in dp:
            return dp[(S, i)]
        if not S:
            return dist_arr[i][0] # Return to the starting city
        
        min_cost = float('inf')
        prev_city = None
        for city in S:
            cost = dist_arr[i][city] + tsp_helper(S - {city}, city)
            if cost < min_cost:
                min_cost = cost
                prev_city = city
        dp[(S, i)] = min_cost
        prev[(S, i)] = prev_city
        return min_cost
    
    best_cost = tsp_helper(frozenset(range(1, n)), root)
    now_city = root
    tour = [now_city]
    S = frozenset(range(1, n))
    while S:
        now_city = prev[(S, now_city)]
        tour.append(now_city)
        S -= {now_city}
    return tour



### Debugging
A simplified verion of the otter tests are pasted here for your convenience. Feel free to add whatever print statements or assertions you'd like when debugging.

_Points:_ 6

In [6]:
# tests on very small cases
for dist_arr, expected_distance in tqdm.tqdm(test_data['q2.1']):
    try:
        result = tsp_dp(dist_arr)
    except Exception as e:
        raise Exception(f"Your function threw an error (see above)") from e
    assert set(result) == set(range(len(dist_arr))), f"Your output does not visit all cities"
    student_length = validate_tour(result, dist_arr)
    assert student_length >= 0, f"Your output is not a valid tour"
    assert student_length == expected_distance, f"Your output is not a minimum distance tour"

100%|██████████| 20/20 [00:15<00:00,  1.32it/s]


In [7]:
grader.check("TSP")

100%|██████████| 16/16 [00:10<00:00,  1.49it/s]


TSP results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit.

In [ ]:
grader.export(pdf=False, force_save=True)